In [1]:
#Description: The purpose of this python report is to build a sentiment anlaysis from end user Ultra Calls 

In [2]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json
import numpy as np
import re
from textblob import TextBlob
import textblob
import tweepy
import datetime, time
import sys
import threading

import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt4
from alpaca_trade_api.rest import REST
%matplotlib inline
#import alpaca_trade_api as api

In [3]:

# authentication and connection details
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = 'https://paper-api.alpaca.markets'

# instantiate REST API
alpaca_api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, base_url, api_version='v2', raw_data=True)

# obtain account information
account = alpaca_api.get_account()

load_dotenv()

True

In [4]:
#Keys used to pull in the Tweepy Account

consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
Access_Token = os.getenv("Access_Token")
Access_Token_Secret = os.getenv("Access_Token_Secret")

In [5]:
# This is to make sure that the limit we have set on our Developer Account does not go over.

auth_handler = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth_handler.set_access_token(Access_Token, Access_Token_Secret)
tweepy_api = tweepy.API(auth_handler, wait_on_rate_limit=True)


### Dictionary for Twitter

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'display_text_range', 'entities', 'favorite', 'favorite_count', 'favorited', 'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'possibly_sensitive', 'retweet', 'retweet_count', 'retweeted', 'retweets', 'source', 'source_url', 'truncated', 'user']

TweepError: Twitter error response: status code = 404 (means that the ID does not exist)

In [6]:
#Variables to build the Tweepy Cursor
tweet_amount = 100
tweets = []
likes = []
time = []


In [7]:
#This Line of code is to clear up the tweets that we are pulling from the user

def clean_tweet(tweetz):
    tweetz = tweetz.replace('@', '')
    tweetz = tweetz.replace('http\S+', '')
    return tweetz

In [8]:
# Loop function to provide the information for the tweets and Like count

records = []
user_id = ['Ultra_Calls']
for user in user_id:
    print(user)
    for i in tweepy.Cursor(tweepy_api.user_timeline, screen_name=user, include_rts=False, tweet_mode='extended').items(tweet_amount):
        record = {}
        record['user'] = user
        record['Tweets'] = clean_tweet(i.full_text)
        record['favorite_count'] = i.favorite_count
        record['created_at'] = i.created_at
        records.append(record)

Ultra_Calls


In [9]:
#have a data frame avaliable for the following code

tweets_df = pd.DataFrame(records)


In [10]:
# This line of code is to pull the stock acronym from the tweet

def find_ticker(word):
    if any(ch.isdigit() for ch in word):
        return None
    else:
        return word

In [11]:
# This is to clear the $ from the stock acronym

tweets_df['Ticker'] = tweets_df['Tweets'].apply(lambda x: [find_ticker(word) for word in x.split() if '$' in word])

In [12]:
# Using the sentiment analysis from textblob, to determine whether the tweet is negative or postive 

tweets_df['Polarity'] = tweets_df['Tweets'].map(lambda tweet: textblob.TextBlob(tweet).sentiment.polarity)
tweets_df['Result'] = tweets_df['Polarity'].map(lambda pol: '+' if pol > 0 else '-')
positive = tweets_df[tweets_df.Result == '+'].count()['Tweets']
negative = tweets_df[tweets_df.Result == '-'].count()['Tweets']


In [13]:
# As a group we determined that a postive vaule of .3 would provide the ideal returns.

tweets_df['buy'] = tweets_df['Polarity'] >= .3
tweets_df

,user,Tweets,favorite_count,created_at,Ticker,Polarity,Result,buy
0,Ultra_Calls,camicam_1 Booty break,0,2021-08-06 17:22:43,[],0.000000,-,False
1,Ultra_Calls,If you ever see me say HI 👋,67,2021-08-06 17:17:29,[],0.000000,-,False
2,Ultra_Calls,21 SVJ ROADSTER 🍑 https://t.co/TIf6NHHfhp,152,2021-08-06 17:14:43,[],0.000000,-,False
3,Ultra_Calls,Reformed_Trader It’s Bitcoin/Blockchain $PIXY,2,2021-08-06 16:51:18,[$PIXY],0.000000,-,False
4,Ultra_Calls,"MahsaSaderyoun Yes 300,000 $ECOR shares",6,2021-08-06 16:18:38,[$ECOR],0.000000,-,False
...,...,...,...,...,...,...,...,...
95,Ultra_Calls,$WHLR NHOD holding full through bell 🔔 too str...,65,2021-08-04 08:40:23,[$WHLR],0.391667,+,True
96,Ultra_Calls,$WHLR 4.15 huge breakout spot,39,2021-08-04 08:20:19,[$WHLR],0.400000,+,True
97,Ultra_Calls,$WHLR Already huge volume to start,49,2021-08-04 08:17:38,[$WHLR],0.400000,+,True
98,Ultra_Calls,$WHLR Watch and learn at opening bell. Massive...,55,2021-08-04 08:08:58,[$WHLR],0.000000,-,False


In [14]:
#This line of code cleans up the stock acrynom so we can have it ready for the Alpaca API

def clean_ticker(row):
    tickers = [ticker for ticker in row if ticker != None]
    return tickers


In [15]:
#Panda dataframe to be used for the Alpaca API

tweets_df['buy_list'] = tweets_df['Ticker'].apply(lambda row: clean_ticker(row))
tweets_df

,user,Tweets,favorite_count,created_at,Ticker,Polarity,Result,buy,buy_list
0,Ultra_Calls,camicam_1 Booty break,0,2021-08-06 17:22:43,[],0.000000,-,False,[]
1,Ultra_Calls,If you ever see me say HI 👋,67,2021-08-06 17:17:29,[],0.000000,-,False,[]
2,Ultra_Calls,21 SVJ ROADSTER 🍑 https://t.co/TIf6NHHfhp,152,2021-08-06 17:14:43,[],0.000000,-,False,[]
3,Ultra_Calls,Reformed_Trader It’s Bitcoin/Blockchain $PIXY,2,2021-08-06 16:51:18,[$PIXY],0.000000,-,False,[$PIXY]
4,Ultra_Calls,"MahsaSaderyoun Yes 300,000 $ECOR shares",6,2021-08-06 16:18:38,[$ECOR],0.000000,-,False,[$ECOR]
...,...,...,...,...,...,...,...,...,...
95,Ultra_Calls,$WHLR NHOD holding full through bell 🔔 too str...,65,2021-08-04 08:40:23,[$WHLR],0.391667,+,True,[$WHLR]
96,Ultra_Calls,$WHLR 4.15 huge breakout spot,39,2021-08-04 08:20:19,[$WHLR],0.400000,+,True,[$WHLR]
97,Ultra_Calls,$WHLR Already huge volume to start,49,2021-08-04 08:17:38,[$WHLR],0.400000,+,True,[$WHLR]
98,Ultra_Calls,$WHLR Watch and learn at opening bell. Massive...,55,2021-08-04 08:08:58,[$WHLR],0.000000,-,False,[$WHLR]


In [16]:
# This is the Alpaca API code

def buy_ticker(ticker):
    print("using alpacha....")
    symbol_bars = alpaca_api.get_barset(ticker, 'minute', 1).df.iloc[0]
    symbol_price = symbol_bars[ticker]['close']
    alpaca_api.submit_order(
    symbol=ticker,
    qty=50,
    side='buy',
    type='market',
    time_in_force='gtc',
    order_class='bracket',
    stop_loss = {
        'stop_price': symbol_price * 0.95,
        'limit_price':  symbol_price * 0.95
    },
    take_profit={'limit_price': (symbol_price) * 1.10}
    )
    print(f"Just bought {ticker}...")

In [18]:
# This is to show the results of our stop, limit prices of the stocks that are acceptable in our sentiment analysis

for _, tweet in tweets_df.iterrows():
    if tweet['buy'] == True:
        for ticker in tweet['buy_list']:
            ds = tweet['created_at']
            user = tweet['user']
            print(f"buy {ticker} at {ds} based on signal from {user}")
            try:
                buy_ticker(ticker[1:])
            except:
                print(f"skipping {ticker}...at {ds}")
                print()
                

buy $OCUP at 2021-08-06 13:55:01 based on signal from Ultra_Calls
using alpacha....
Just bought OCUP...
buy $OCUP at 2021-08-05 20:31:28 based on signal from Ultra_Calls
using alpacha....
Just bought OCUP...
buy $BAOS at 2021-08-05 16:23:09 based on signal from Ultra_Calls
using alpacha....
Just bought BAOS...
buy $ENSC at 2021-08-05 11:55:16 based on signal from Ultra_Calls
using alpacha....
Just bought ENSC...
buy $TLGT at 2021-08-05 01:36:08 based on signal from Ultra_Calls
using alpacha....
Just bought TLGT...
buy $TLGT at 2021-08-04 16:55:28 based on signal from Ultra_Calls
using alpacha....
Just bought TLGT...
buy $TLGT at 2021-08-04 15:51:13 based on signal from Ultra_Calls
using alpacha....
Just bought TLGT...
buy $WHLR at 2021-08-04 13:42:36 based on signal from Ultra_Calls
using alpacha....
Just bought WHLR...
buy $WHLR at 2021-08-04 13:36:26 based on signal from Ultra_Calls
using alpacha....
Just bought WHLR...
buy $WHLR at 2021-08-04 08:40:23 based on signal from Ultra_Call

In [21]:
alpaca_activities = alpaca_api.get_activities()
activities_df = pd.DataFrame(alpaca_activities)

In [23]:
activities_df.to_excel("my_act.xlsx")